    Prepare

In [ ]:
# Libraries

import datetime
import contextlib
from io import StringIO

import pandas as pd
import numpy as np
from garminconnect import Garmin

import gspread
from google.oauth2.service_account import Credentials

# https://medium.com/@datascienceandvisualisation/a-walkthrough-of-downloading-and-analysing-garmin-connect-data-using-python-b38c51f2fd85
# https://github.com/cyberjunky/python-garminconnect/blob/master/example.py
# https://medium.com/@azholud/analysis-and-visualization-of-activities-from-garmin-connect-b3e021c62472

In [88]:
# Import help functions

import pandas as pd
import numpy as np

import datetime
from garminconnect import Garmin
import gspread

import contextlib
from io import StringIO

import config
from helpFunctions import clean_data 

In [3]:
# Authenticate

print("1. Authentication ...")

# Garmin API
print("Authenticating Garmin Connect API for {} ...".format(config.GARMIN_EMAIL))
try:
    garminClient = Garmin(config.GARMIN_EMAIL, config.GARMIN_PASSWORD)
    garminClient.login()
    print("~> Login to Garmin Connect API successful! :)")
except Exception as e:
    print("~> Error logging to Garmin Connect API: {} :(".format(e))

# Google drive API
print("\nAuthenticating Google Drive API ...")
try:
    googleDrive_client = gspread.authorize(config.DRIVE_CREDENTIALS)
    print("~> Authentication to Google Drive API successful! :)")
except Exception as e:
    print("~> Error authenticating to Google Drive API: {} :(".format(e))

1. Authentication ...
Authenticating Garmin Connect API for pecek.urh@gmail.com ...
~> Login to Garmin Connect API successful! :)

Authenticating Google Drive API ...
~> Authentication to Google Drive API successful! :)


In [4]:
# Prepare drive file to write into

# Daily Logs
print("\n2. Opening and preparing Daily Log file {} ...".format(config.DRIVE_DAILY_LOG_FILENAME))
try:
    daily_log_file = googleDrive_client.open(config.DRIVE_DAILY_LOG_FILENAME)
    daily_log_sheet = daily_log_file.get_worksheet(0)
    daily_log_data = daily_log_sheet.get_all_values()
    daily_log_df = pd.DataFrame(daily_log_data[1:], columns=daily_log_data[0]) 
    print("~> Daily Log file succesfully imported and available for formating! :)")
except Exception as e:
    print("Error opening Daily Log: {}".format(e))

# TP Logs
print("\n3. Opening and preparing TP Log file {} ...".format(config.DRIVE_TP_LOG_FILENAME))
try:
    tp_log_file = googleDrive_client.open(config.DRIVE_TP_LOG_FILENAME)
    tp_log_sheet = tp_log_file.get_worksheet(0)
    tp_log_data = tp_log_sheet.get_all_values()
    tp_log_df = pd.DataFrame(tp_log_data[1:], columns=tp_log_data[0]) 
    print("~> TP Log file succesfully imported and available for formating! :)")
except Exception as e:
    print("Error opening TP Log: {}".format(e))


2. Opening and preparing Daily Log file dailySandboxLog_Urh ...
~> Daily Log file succesfully imported and available for formating! :)

3. Opening and preparing TP Log file trainingPeaksSandboxLog_Urh ...
~> TP Log file succesfully imported and available for formating! :)


In [89]:
# Reload libraries
import importlib 

import config
importlib.reload(config)

import singleDay_dailyStats
importlib.reload(singleDay_dailyStats)
from singleDay_dailyStats import singleDay_dailyStats

import singleDay_activityStats
importlib.reload(singleDay_activityStats)
from singleDay_activityStats import singleDay_activityStats

In [90]:
# Calculate and write daily statistics to Drive sheet ~ Day by day
start_date = datetime.datetime(2024, 9, 13).date() 
end_date = datetime.date.today() - datetime.timedelta(days=1)
# start_date = datetime.datetime(2025, 2, 8).date() 
# end_date = datetime.datetime(2025, 2, 8).date() 
date_list = [(start_date + datetime.timedelta(days=i)) for i in range((end_date - start_date).days + 1)]

for singleDate in date_list:

    print("\n4. Prepare and write daily statistics and activity statistics ~ Single day ~> {} ...".format(singleDate))

    # print("4.1 Daily statistics")
    # print("~> Prepare")
    # singleDay_dailyStats_dict = singleDay_dailyStats(garminClient, singleDate)
    # print("~> Write into Daily Log file")
    # if not daily_log_sheet.row_values(1):
    #     daily_log_sheet.insert_row(config.DAILY_LOG_EXPECTED_HEADERS, index=1)
    # with contextlib.redirect_stdout(StringIO()):
    #     daily_log_raw = clean_data(singleDay_dailyStats_dict)
    #     daily_log_df = pd.DataFrame([daily_log_raw])
    #     daily_log_sheetFormat = daily_log_df.values.tolist()
    #     daily_log_sheet.append_rows(daily_log_sheetFormat)  

    print("4.2 Activity statistics")
    print("~> Prepare")
    singleDay_activityStats_dict = singleDay_activityStats(garminClient, singleDate)
    print("~> Write into Training Peaks Log file")
    if not tp_log_sheet.row_values(1):
        tp_log_sheet.insert_row(config.TP_LOG_EXPECTED_HEADERS, index=1)
    for i in reversed(range(len(singleDay_activityStats_dict))):
        with contextlib.redirect_stdout(StringIO()):
            tp_log_raw = clean_data(singleDay_activityStats_dict["activity_{}".format(i)])
            tp_log_df = pd.DataFrame([tp_log_raw])
            tp_log_sheetFormat = tp_log_df.values.tolist()
            tp_log_sheet.append_rows(tp_log_sheetFormat)


4. Prepare and write daily statistics and activity statistics ~ Single day ~> 2024-09-13 ...
4.2 Activity statistics
~> Prepare
~> Write into Training Peaks Log file

4. Prepare and write daily statistics and activity statistics ~ Single day ~> 2024-09-14 ...
4.2 Activity statistics
~> Prepare
~> Write into Training Peaks Log file

4. Prepare and write daily statistics and activity statistics ~ Single day ~> 2024-09-15 ...
4.2 Activity statistics
~> Prepare
~> Write into Training Peaks Log file

4. Prepare and write daily statistics and activity statistics ~ Single day ~> 2024-09-16 ...
4.2 Activity statistics
~> Prepare
~> Write into Training Peaks Log file

4. Prepare and write daily statistics and activity statistics ~ Single day ~> 2024-09-17 ...
4.2 Activity statistics
~> Prepare
~> Write into Training Peaks Log file

4. Prepare and write daily statistics and activity statistics ~ Single day ~> 2024-09-18 ...
4.2 Activity statistics
~> Prepare
~> Write into Training Peaks Log fil